# Python

![](https://leukipp.github.io/xmastree/kernelspecs/python.png)

In [ ]:
import os
import js
import urllib

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

from pyodide_js import FS
from IPython.display import HTML, Javascript


## Utils

In [ ]:
class FileSystemUtils(object):
    def __init__(self, fs, path):
        """ indexedDB filesystem storage """
        self.fs = fs
        self.path = path
        self.type = self.fs.filesystems.IDBFS

        self.mount()
        self.upsync()

    def mount(self):
        """ mount kernel filesystem """
        if not os.path.exists(self.path):
            self.fs.mkdir(self.path)
            self.fs.mount(self.type, {}, self.path)

    def upsync(self):
        """ sync from browser to kernel filesystem """
        self.fs.syncfs(True, lambda x: print(x) if x else None)

    def downsync(self):
        """ sync from kernel to browser filesystem """
        self.fs.syncfs(False, lambda x: print(x) if x else None)


In [ ]:
class Simulator(object):
    def __init__(self, width='100%', height='600px'):
        """ three js simulator """
        self.width = width
        self.height = height

        self.id = 'simulator'
        self.root = '/xmastree/files/simulator/index.html'
        self.settings = {'material.background': '0x2f2a46'}

    def src(self):
        return f'{self.root}#{urllib.parse.urlencode(self.settings)}'

    def render(self):
        """ render simulator html """
        layout = {
            'id': self.id,
            'frameborder': '0',
            'width': self.width,
            'height': self.height,
            'src': self.src()
        }
        attributes = ' '.join([f'{k}="{v}"' for k, v in layout.items()])
        return HTML(f'<iframe allowfullscreen {attributes} />')

    def update(self, **kwargs):
        """ update simulator settings """
        for k, v in kwargs.items():
            self.settings[k] = str(v)
        return Javascript(f'document.getElementById("{self.id}").src="{self.src()}"')


### Tests

In [ ]:
home = os.path.expanduser('~')
data = os.path.join(home, 'data')

# init filesystem
fs = FileSystemUtils(FS, data)


In [ ]:
# create some file
f = open(os.path.join(data, 'test3.csv'), 'w')
f.write('3,123,äbc')
f.close()


In [ ]:
# read some file
f = open(os.path.join(data, 'test3.csv'), 'r')
f.read()


In [ ]:
# sync from kernel to browser filesystem
fs.downsync()


In [ ]:
# init simulator
sim = Simulator()


In [ ]:
# render simulator
sim.render()


In [ ]:
# update simulator settings
sim.update(fps=30, loop='true')


In [ ]:
# fetch csv
result = await js.fetch('https://raw.githubusercontent.com/standupmaths/xmastree2021/main/examples/bouncy-ball.csv')
csv = await result.text()

# save csv
path = os.path.join(data, 'bouncy-ball.csv')
with open(path, 'w') as file:
    file.write(csv)
fs.downsync()


## Matplotlib

In [ ]:
x = np.linspace(0, 10, 1000)

plt.plot(x, np.sin(x))
plt.show()


## Widgets

In [ ]:
import micropip

await micropip.install('ipympl')
await micropip.install('ipywidgets')


In [ ]:
from ipywidgets import IntSlider, IntText, link


In [ ]:
%matplotlib widget


In [ ]:
x = np.linspace(0, 10, 1000)
plt.plot(x, np.sin(x))


In [ ]:
slider = IntSlider()
slider


In [ ]:
text = IntText()
text


In [ ]:
link((slider, 'value'), (text, 'value'))
